# Install and init DVC

Prerequisites: 
-  DVC and requirements.txt packages installed (if not - check README.md file for instructions)
-  A project repository is a Git repo 

## Install with pip

In [1]:
!pip install dvc==1.8.4

## Checkout branch `experiments`

In [1]:
!git checkout -b experiments

fatal: A branch named 'experiments' already exists.


## Initialize DVC

References: 
- https://dvc.org/doc/get-started/initialize 

In [2]:
!dvc init

ERROR: failed to initiate DVC - '.dvc' exists. Use `-f` to force.


## Commit changes

In [3]:
%%bash

git add .
git commit -m "Initialize DVC"

[experiments b20e564] Initialize DVC
 577 files changed, 209775 insertions(+), 26 deletions(-)
 create mode 100644 .jupyter/bin/Activate.ps1
 create mode 100644 .jupyter/bin/activate
 create mode 100644 .jupyter/bin/activate.csh
 create mode 100644 .jupyter/bin/activate.fish
 create mode 100755 .jupyter/bin/pip
 create mode 100755 .jupyter/bin/pip3
 create mode 100755 .jupyter/bin/pip3.9
 create mode 120000 .jupyter/bin/python
 create mode 120000 .jupyter/bin/python3
 create mode 120000 .jupyter/bin/python3.9
 create mode 100644 .jupyter/lib/python3.9/site-packages/_distutils_hack/__init__.py
 create mode 100644 .jupyter/lib/python3.9/site-packages/_distutils_hack/override.py
 create mode 100644 .jupyter/lib/python3.9/site-packages/distutils-precedence.pth
 create mode 100644 .jupyter/lib/python3.9/site-packages/pip-21.2.3.dist-info/INSTALLER
 create mode 100644 .jupyter/lib/python3.9/site-packages/pip-21.2.3.dist-info/LICENSE.txt
 create mode 100644 .jupyter/lib/python3.9/site-package

## Review Files and Directories created by DVC

In [4]:
!ls -a .dvc 

.  ..  config  .gitignore  plots  tmp


In [10]:
!cat .dvc/.gitignore

/config.local
/tmp
/cache


# Quick Tour of DVC features

## Data Versioning

In [6]:
# Get data 

import pandas as pd
from sklearn.datasets import load_iris

data = load_iris(as_frame=True)
list(data.target_names)
data.frame.to_csv('data/iris.csv', index=False)

ModuleNotFoundError: No module named 'pandas'

In [9]:
# Look on data

data.frame.head()

NameError: name 'data' is not defined

In [8]:
%%bash

du -sh data/*

du: cannot access 'data/*': No such file or directory


CalledProcessError: Command 'b'\ndu -sh data/*\n'' returned non-zero exit status 1.

## Add file under DVC control

In [7]:
%%bash

dvc add data/iris.csv

ERROR: output 'data/iris.csv' does not exist


CalledProcessError: Command 'b'\ndvc add data/iris.csv\n'' returned non-zero exit status 1.

In [11]:
!du -sh data/*

4,0K	data/iris.csv
4,0K	data/iris.csv.dvc


In [12]:
!git status -s data/

?? data/.gitignore
?? data/iris.csv.dvc


In [13]:
%%bash

git add .
git commit -m "Add a source dataset"

[experiments 40d1734] Add a source dataset
 2 files changed, 4 insertions(+)
 create mode 100644 data/.gitignore
 create mode 100644 data/iris.csv.dvc


### What is DVC-file?

Data file internals


>    If you take a look at the DVC-file, you will see that only outputs are defined in outs. 
    In this file, only one output is defined. The output contains the data file path in the repository and md5 cache.
    This md5 cache determines a location of the actual content file in DVC cache directory .dvc/cache
    >> Output from DVC-files defines the relationship between the data file path in a repository and the path in a cache directory. See also DVC File Format



(c) dvc.org https://dvc.org/doc/tutorial/define-ml-pipeline

In [14]:
!cat data/iris.csv.dvc

outs:
- md5: 4d301abed5efe50eccda350cde38e0eb
  path: iris.csv


## Create and Reproduce ML pipelines 

Stages 
- extract features 
- split dataset 
- train 
- evaluate 


### Add a pipeline stage with 'dvc run'

In [15]:
!dvc run -n feature_extraction \
    -d src/featurization.py \
    -d data/iris.csv \
    -o data/iris_featurized.csv \
    python src/featurization.py

Running stage 'feature_extraction' with command:                      core>
	python src/featurization.py
Creating 'dvc.yaml'                                                   core>
Adding stage 'feature_extraction' in 'dvc.yaml'
Generating lock file 'dvc.lock'
Updating lock file 'dvc.lock'

To track the changes with git, run:

	git add dvc.yaml data/.gitignore dvc.lock


In [16]:
!ls 

data			       dvc.lock  dvc.yaml   requirements.txt
dvc-1-get-started.ipynb.ipynb  dvc-venv  README.md  src


In [17]:
!cat dvc.yaml

stages:
  feature_extraction:
    cmd: python src/featurization.py
    deps:
    - data/iris.csv
    - src/featurization.py
    outs:
    - data/iris_featurized.csv


In [18]:
import pandas as pd

features = pd.read_csv('data/iris_featurized.csv')
features.head()

,sepal_length,sepal_width,petal_length,petal_width,target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [19]:
!git status -s

 M data/.gitignore
?? dvc.lock
?? dvc.yaml


In [20]:
%%bash
git add .
git commit -m "Add stage features_extraction"

[experiments 5f733fe] Add stage features_extraction
 3 files changed, 19 insertions(+)
 create mode 100644 dvc.lock
 create mode 100644 dvc.yaml


### Add split train/test stage (via dvc.yaml)

In [21]:
!dvc run -n split_dataset \
    -d src/split_dataset.py \
    -d data/iris_featurized.csv \
    -o data/train.csv \
    -o data/test.csv \
    python src/split_dataset.py --test_size 0.4

Running stage 'split_dataset' with command:                           core>
	python src/split_dataset.py --test_size 0.4
Adding stage 'split_dataset' in 'dvc.yaml'                            core>
Updating lock file 'dvc.lock'

To track the changes with git, run:

	git add dvc.yaml data/.gitignore dvc.lock


In [22]:
!cat dvc.yaml

stages:
  feature_extraction:
    cmd: python src/featurization.py
    deps:
    - data/iris.csv
    - src/featurization.py
    outs:
    - data/iris_featurized.csv
  split_dataset:
    cmd: python src/split_dataset.py --test_size 0.4
    deps:
    - data/iris_featurized.csv
    - src/split_dataset.py
    outs:
    - data/test.csv
    - data/train.csv


### Add train stage

In [23]:
!dvc run -n train \
    -d src/train.py \
    -d data/train.csv \
    -o data/model.joblib \
    python src/train.py

Running stage 'train' with command:                                   core>
	python src/train.py
Adding stage 'train' in 'dvc.yaml'                                    core>
Updating lock file 'dvc.lock'

To track the changes with git, run:

	git add dvc.lock dvc.yaml data/.gitignore


In [24]:
!cat dvc.yaml

stages:
  feature_extraction:
    cmd: python src/featurization.py
    deps:
    - data/iris.csv
    - src/featurization.py
    outs:
    - data/iris_featurized.csv
  split_dataset:
    cmd: python src/split_dataset.py --test_size 0.4
    deps:
    - data/iris_featurized.csv
    - src/split_dataset.py
    outs:
    - data/test.csv
    - data/train.csv
  train:
    cmd: python src/train.py
    deps:
    - data/train.csv
    - src/train.py
    outs:
    - data/model.joblib


### Add evaluate stage

In [25]:
!dvc run -n evaluate \
    -d src/train.py \
    -d src/evaluate.py \
    -d data/test.csv \
    -d data/model.joblib \
    -m data/eval.txt \
    python src/evaluate.py

Running stage 'evaluate' with command:                                core>
	python src/evaluate.py
Adding stage 'evaluate' in 'dvc.yaml'                                 core>
Updating lock file 'dvc.lock'

To track the changes with git, run:

	git add dvc.yaml dvc.lock data/.gitignore


In [26]:
!cat dvc.yaml

stages:
  feature_extraction:
    cmd: python src/featurization.py
    deps:
    - data/iris.csv
    - src/featurization.py
    outs:
    - data/iris_featurized.csv
  split_dataset:
    cmd: python src/split_dataset.py --test_size 0.4
    deps:
    - data/iris_featurized.csv
    - src/split_dataset.py
    outs:
    - data/test.csv
    - data/train.csv
  train:
    cmd: python src/train.py
    deps:
    - data/train.csv
    - src/train.py
    outs:
    - data/model.joblib
  evaluate:
    cmd: python src/evaluate.py
    deps:
    - data/model.joblib
    - data/test.csv
    - src/evaluate.py
    - src/train.py
    metrics:
    - data/eval.txt


### Reproduce pipeline

In [27]:
!dvc repro split_dataset

'data/iris.csv.dvc' didn't change, skipping                           core>
Stage 'feature_extraction' didn't change, skipping
Stage 'split_dataset' didn't change, skipping
Data and pipelines are up to date.


## Collaborate on ML Experiments 

### Specify remote storage (local ~ /tmp/dvc)


In [28]:
!dvc remote add -d local /tmp/dvc

Setting 'local' as a default remote.


### Push features to remote storage

In [29]:
!dvc push

  0% Uploading|                                      |0/2 [00:00<?,     ?file/s]
!
  0%|          |data/model.joblib              0.00/1.01k [00:00<?,       ?it/s]

!

  0%|          |data/eval.txt                       0/124 [00:00<?,       ?it/s]
                                                                                

2 files pushed                                                                  


### Create tag `experiment-1`

In [30]:
!git tag -a experiment-1 -m "experiment-1"

### Checkout into your teammate experiment state

In [31]:
%%bash 

git checkout experiment-1
dvc checkout

M	.dvc/config
M	data/.gitignore
M	dvc-1-get-started.ipynb.ipynb
M	dvc.lock
M	dvc.yaml


Note: checking out 'experiment-1'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

HEAD сейчас на 5f733fe... Add stage features_extraction


### Check Metrics

In [32]:
!dvc metrics show

	data/eval.txt:                                                       core>
		f1_score: 0.7861833464670345


### Reproduce experiment

In [33]:
# Nothing to reproduce
!dvc repro

'data/iris.csv.dvc' didn't change, skipping                           core>
Stage 'feature_extraction' didn't change, skipping
Stage 'split_dataset' didn't change, skipping
Stage 'train' didn't change, skipping
Stage 'evaluate' didn't change, skipping
Data and pipelines are up to date.


In [34]:
!dvc repro -f

Verifying data sources in stage: 'data/iris.csv.dvc'                  core>

Running stage 'feature_extraction' with command:
	python src/featurization.py
                                                                      core>
Running stage 'split_dataset' with command:
	python src/split_dataset.py --test_size 0.4
                                                                      core>
Running stage 'train' with command:
	python src/train.py
                                                                      core>
Running stage 'evaluate' with command:
	python src/evaluate.py
                                                                      core>
To track the changes with git, run:

	git add data/iris.csv.dvc


In [35]:
# Check Metrics

!dvc metrics show

	data/eval.txt:                                                       core>
		f1_score: 0.7861833464670345
